# Instruction

Environment: google colab \
Upload the following files on google drive: \
train.csv \
test.csv \
noisy-images.zip (zip noisy-images folder)


In [ ]:
from __future__ import print_function
import torch
import torchtext
import torchvision.transforms as transforms
from torch import nn, optim
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.models as models
import pandas as pd
import numpy as np
import time

# Import image folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/noisy-images.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: noisy-images/3212.jpg   
  inflating: __MACOSX/noisy-images/._3212.jpg  
  inflating: noisy-images/11755.jpg  
  inflating: __MACOSX/noisy-images/._11755.jpg  
  inflating: noisy-images/52888.jpg  
  inflating: __MACOSX/noisy-images/._52888.jpg  
  inflating: noisy-images/12274.jpg  
  inflating: __MACOSX/noisy-images/._12274.jpg  
  inflating: noisy-images/44032.jpg  
  inflating: __MACOSX/noisy-images/._44032.jpg  
  inflating: noisy-images/14605.jpg  
  inflating: __MACOSX/noisy-images/._14605.jpg  
  inflating: noisy-images/9071.jpg   
  inflating: __MACOSX/noisy-images/._9071.jpg  
  inflating: noisy-images/52650.jpg  
  inflating: __MACOSX/noisy-images/._52650.jpg  
  inflating: noisy-images/6342.jpg   
  inflating: __MACOSX/noisy-images/._6342.jpg  
  inflating: noisy-images/35813.jpg  
  inflating: __MACOSX/noisy-images/._35813.jpg  
  inflating: noisy-images/10463.jpg  
  inflating: __MACOSX/noisy-images/._10463.jpg  
  inflating: noisy-

# Input train and test data

If you want to try validation data, uncomment the code.

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
#train_df, valid_df = train_test_split(train_df, test_size=0.2, random_state=480)

test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create category dictionary

In [ ]:
category_map = {cat: val for val, cat in enumerate(set(train_df['category']))}
print(category_map)
gender_map = {gender: val for val, gender in enumerate(set(train_df['gender']))}
print(gender_map)
color_map = {color: val for val, color in enumerate(set(train_df['baseColour']))}
print(color_map)
season_map = {season: val for val, season in enumerate(set(train_df['season']))}
print(season_map)
usage_map = {use: val for val, use in enumerate(set(train_df['usage']))}
print(usage_map)

{'Lips': 0, 'Topwear': 1, 'Watches': 2, 'Free Gifts': 3, 'Accessories': 4, 'Makeup': 5, 'Nails': 6, 'Sandal': 7, 'Headwear': 8, 'Bags': 9, 'Innerwear': 10, 'Flip Flops': 11, 'Saree': 12, 'Ties': 13, 'Cufflinks': 14, 'Jewellery': 15, 'Scarves': 16, 'Belts': 17, 'Socks': 18, 'Dress': 19, 'Apparel Set': 20, 'Bottomwear': 21, 'Shoes': 22, 'Loungewear and Nightwear': 23, 'Fragrance': 24, 'Eyewear': 25, 'Wallets': 26}
{'Women': 0, 'Girls': 1, 'Unisex': 2, 'Boys': 3, 'Men': 4}
{'Steel': 0, 'Red': 1, 'Bronze': 2, 'Turquoise Blue': 3, 'Cream': 4, 'Gold': 5, 'Yellow': 6, 'Silver': 7, 'Sea Green': 8, 'Lavender': 9, 'Blue': 10, 'Olive': 11, 'Off White': 12, 'Lime Green': 13, 'Tan': 14, 'Orange': 15, 'Burgundy': 16, 'Khaki': 17, 'Rust': 18, 'Copper': 19, 'Mushroom Brown': 20, 'Beige': 21, 'Black': 22, 'Navy Blue': 23, 'Fluorescent Green': 24, 'Charcoal': 25, 'Grey Melange': 26, 'Metallic': 27, 'Nude': 28, 'Skin': 29, 'Coffee Brown': 30, 'Teal': 31, 'Peach': 32, 'Purple': 33, 'Multi': 34, 'Magenta':

In [ ]:
max_words = 17

# Build Vocabulary

In [ ]:
tokenizer = get_tokenizer(tokenizer='spacy')
train_iter = iter(train_df['noisyTextDescription'])

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

vocab_size = len(vocab)
print(vocab_size)

9054


# Image transformation

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

# Input encoding

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
gender_pipeline = lambda x: gender_map.get(x)
color_pipeline = lambda x: color_map.get(x)
season_pipeline = lambda x: season_map.get(x)
use_pipeline = lambda x: usage_map.get(x)
cat_pipeline = lambda x: category_map.get(x)

# Format input and output

credit: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
class TextImageDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list
        self.transform = transform
        self.translate()
        
    def translate(self):
        self.data_list.noisyTextDescription = [text_pipeline(text) for text in self.data_list.noisyTextDescription]
        self.data_list.noisyTextDescription = [text + [0]*(max_words-len(text)) for text in self.data_list.noisyTextDescription]
        self.data_list.gender = [gender_pipeline(gender) for gender in self.data_list.gender]
        self.data_list.baseColour = [color_pipeline(color) for color in self.data_list.baseColour]
        self.data_list.season = [season_pipeline(season) for season in self.data_list.season]
        self.data_list.usage = [use_pipeline(use) for use in self.data_list.usage]
        self.data_list.category = [cat_pipeline(cat) for cat in self.data_list.category]

    def __len__(self):
        return len(self.data_list.category)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        id = self.data_list['id'].iloc[idx]
        image_path = f'/content/noisy-images/{id}.jpg'
        image = Image.open(image_path)
        if self.transform is not None:
            image = self.transform(image)

        text = torch.tensor(self.data_list['noisyTextDescription'].iloc[idx])
        gender = self.data_list['gender'].iloc[idx]
        color = self.data_list['baseColour'].iloc[idx]
        season = self.data_list['season'].iloc[idx]
        usage = self.data_list['usage'].iloc[idx]
        category = self.data_list['category'].iloc[idx]
        
        return text, gender, color, season, usage, image, category

# Model

###Input:
a combination of noisy text, noisy image, and other categorical variables (gender, color, season, and usage)

###Output:
a prediction for 27 classes.

The model architecture consists of several layers:

###Text layer:
Embeds the input text using an embedding layer, applies convolutional filters of size [3, 4, 5], and concatenates the outputs. \
(I used CNN instead of RNN because CNN achieved a higher accuracy)

###Categorical layers:
Embed the corresponding categorical input variables using an embedding layer and flatten the output.

###Image layer:
Applies a sequence of convolutional and pooling layers to the input image and flattens the output.

###Output layer:
Concatenates all the previous outputs, applies dropout regularization (prevent overfitting), and applies a fully connected layer followed by a softmax activation function to output the predicted class probabilities.

In [ ]:
class Model(nn.Module):
    def __init__(self, vocab_size=vocab_size, embedding_dim=100, num_class=27, num_filters=100, filter_sizes=[3,4,5], dropout_prob=0.25):
        super(Model, self).__init__()

        # Text layer
        self.text_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, num_filters, (fs, embedding_dim)) for fs in filter_sizes])

        # Gender layer
        self.gender_embedding = nn.Embedding(5, num_filters)

        # Color layer
        self.color_embedding = nn.Embedding(46, num_filters)

        # Season layer
        self.season_embedding = nn.Embedding(4, num_filters)

        # Usage layer
        self.usage_embedding = nn.Embedding(7, num_filters)

        # Image layer
        self.image_cnn = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(in_features=2240, out_features=256),
            nn.ReLU(),
        )
        
        # Output layer
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(len(filter_sizes) * num_filters + 256 + num_filters * 4, num_class)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, text, gender, color, season, usage, image):
        # Text input forward
        text_embedded = self.text_embedding(text)
        text_embedded = text_embedded.unsqueeze(1)
        text_conv_outputs = []
        for conv in self.convs:
            conv_output = nn.functional.relu(conv(text_embedded))
            conv_output = nn.functional.max_pool2d(conv_output, (conv_output.shape[2], 1))
            text_conv_outputs.append(conv_output.squeeze())
        text_output = torch.cat(text_conv_outputs, dim=1)

        # Gender input forward
        gender_embedded = self.gender_embedding(gender)
        gender_output = gender_embedded.squeeze(1)

        # Color input forward
        color_embedded = self.color_embedding(color)
        color_output = color_embedded.squeeze(1)

        # Season input forward
        season_embedded = self.season_embedding(season)
        season_output = season_embedded.squeeze(1)

        # Usage input forward
        usage_embedded = self.usage_embedding(usage)
        usage_output = usage_embedded.squeeze(1)

        # Image input forward
        image_output = self.image_cnn(image)

        # Output
        output = torch.cat([text_output, image_output, gender_output, color_output, season_output, usage_output], dim=1)
        output = self.dropout(output)
        output = self.softmax(self.fc(output))
        return output

# Train, Evaluate and Test

In [ ]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 200
    start_time = time.time()

    for idx, (text, gender, color, season, usage, image, label) in enumerate(dataloader):
        text, gender, color, season, usage, image, label = text.to(device), gender.to(device), color.to(device), season.to(device), usage.to(device), image.to(device), label.to(device)
        optimizer.zero_grad()
        predicted_label = model(text, gender, color, season, usage, image)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(model, dataloader, criterion):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, gender, color, season, usage, image, label) in enumerate(dataloader):
            text, gender, color, season, usage, image, label = text.to(device), gender.to(device), color.to(device), season.to(device), usage.to(device), image.to(device), label.to(device)
            predicted_label = model(text, gender, color, season, usage, image)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

def test(model, dataloader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for idx, (text, gender, color, season, usage, image, label) in enumerate(dataloader):
            text, gender, color, season, usage, image, label = text.to(device), gender.to(device), color.to(device), season.to(device), usage.to(device), image.to(device), label.to(device)
            predicted_label = model(text, gender, color, season, usage, image)
            predictions += predicted_label.argmax(1).tolist()
    return predictions

# Start

In [ ]:
EPOCHS = 10 # epoch
BATCH_SIZE = 64 # batch size for training

def fit(train_df, valid_df):
  # Define the model, loss function, and optimizer
  model = Model().to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
  total_accu = None
  train_dataset = TextImageDataset(train_df, transform=transform)
  valid_dataset = TextImageDataset(valid_df, transform=transform)
  train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
  valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)
  for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(model, train_loader, optimizer, criterion)
    accu_val = evaluate(model, valid_loader, criterion)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | ''valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, accu_val))
    print('-' * 59)
  return model

# Bagging

###The Ensemble learning technique I used is Bagging.
###Train 10 weak models and make the final prediction using majority voting.

In [ ]:
n_estimators = 10

# Store all models
all_models = []

for i in range(n_estimators):
    # Randomly split the training data to train and validation dataset
    train_df_subset, valid_df_subset = train_test_split(train_df, test_size=0.2, random_state=i)

    print(f"Model {i}:")
    model = fit(train_df_subset, valid_df_subset)
    all_models.append(model)

Model 0:
|   200/  271 batches | accuracy    0.741
-----------------------------------------------------------
| end of epoch   1 | time: 21.98s | valid accuracy    0.866 
-----------------------------------------------------------
|   200/  271 batches | accuracy    0.886
-----------------------------------------------------------
| end of epoch   2 | time: 23.00s | valid accuracy    0.884 
-----------------------------------------------------------
|   200/  271 batches | accuracy    0.915
-----------------------------------------------------------
| end of epoch   3 | time: 22.00s | valid accuracy    0.888 
-----------------------------------------------------------
|   200/  271 batches | accuracy    0.943
-----------------------------------------------------------
| end of epoch   4 | time: 22.50s | valid accuracy    0.886 
-----------------------------------------------------------
|   200/  271 batches | accuracy    0.973
---------------------------------------------------------

# Decoding category

In [ ]:
def get_key(val):
    for key, value in category_map.items():
        if value == val:
            return key

# Validation accuracy

If you want to try validation accuracy, uncomment the code at the data import section.

In [ ]:
valid_dataset = TextImageDataset(valid_df, transform=transform)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

valid_preds = []
for mod in all_models:
    valid_pred = test(mod, valid_loader)
    valid_preds.append(valid_pred)
    
# Combine the predictions using majority voting
valid_preds = np.array(valid_preds)
valid_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=valid_preds)
accuracy = (valid_pred == valid_df.category).sum() / len(valid_df)
print(accuracy)

0.903837263060564


# Validation accuracy
0.903837263060564

# Predict test data

In [ ]:
test_df['category'] = 'Accessories' # Dummy
test_dataset = TextImageDataset(test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

test_preds = []
for mod in all_models:
    test_pred = test(mod, test_loader)
    test_preds.append(test_pred)
    
# Combine the predictions using majority voting
test_preds = np.array(test_preds)
test_pred = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=0, arr=test_preds)

In [ ]:
predict = pd.DataFrame({'id': test_df.id})
predict['category'] = test_pred
predict['category'] = [get_key(value) for value in predict['category']]
predict

,id,category
0,55581,Bottomwear
1,8832,Sandal
2,29112,Bags
3,30961,Shoes
4,49668,Wallets
...,...,...
21623,17255,Topwear
21624,39606,Shoes
21625,41829,Wallets
21626,27416,Topwear


# Download prediction

In [ ]:
predict.to_csv('predict.csv', index=False)